In [1]:
import pandas as pd
import numpy as np
import gc # We're gonna be clearing memory a lot
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train=pd.read_csv("Complete_train.csv")
test=pd.read_csv("test-data.csv")


# 1    0.993864
# 2    0.006136

In [3]:
impression=pd.read_csv("Complete_Impression.csv")

/Users/sowmiyanmorri/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
impression.dtypes

impression_id            object
geo                     float64
site_id                   int64
uuid                     object
adunit_id                 int64
refrenceUrl              object
timestamp_impression     object
uvh                      object
dtype: object

In [5]:
impression.isnull().sum()*1.0

impression_id            2563614.0
geo                      2495010.0
site_id                        0.0
uuid                           0.0
adunit_id                      0.0
refrenceUrl              2570059.0
timestamp_impression           0.0
uvh                     10894611.0
dtype: float64

In [6]:
854538+854540+854536

2563614

In [7]:
impression.nunique()

impression_id           30123717
geo                           40
site_id                      331
uuid                     6343104
adunit_id                    837
refrenceUrl               360869
timestamp_impression    30852318
uvh                      4411362
dtype: int64

In [3]:
print test.shape
print test.nunique()

(188798, 2)
impression_id    58879
item_id           5167
dtype: int64


In [8]:
train.nunique()

impression_id    7664607
item_id             6103
click                  2
dtype: int64

In [9]:
submission=pd.read_csv("sample-submission.csv")
submission.click=1
submission.head()

,impression_id,item_id,click
0,00023151-2f1e-429b-89a5-2f360dad9880-10nql,17066860,1
1,00023151-2f1e-429b-89a5-2f360dad9880-10nql,17277475,1
2,00023151-2f1e-429b-89a5-2f360dad9880-10nql,17325528,1
3,00023151-2f1e-429b-89a5-2f360dad9880-10nql,17325531,1
4,0004fb27-7548-46b6-9db7-666c2d9d7def-10wx4,17052046,1


In [10]:
submission.to_csv("sub_1.csv",index=False)

#Check if impression_ids with no click and click are exclusive set

This will help me to identify if these are exclusive...what are the impression attributes contributing to click

In [11]:
train.impression_id.nunique()

7664607

In [12]:
#Check if impression_ids with no click and click are exclusive set
imp_c=train.impression_id[train.click==2]
imp_nc=train.impression_id[train.click==1]

In [13]:
print len(list(set(imp_c)))#95526 impression id where click occurred
print len(list(set(imp_nc)))#impresssion ids where no click occurred

95526
7660640


In [14]:
print len(list(set(imp_c)-set(imp_nc)))#3966 impression which had only click
print len(list(set(imp_nc)-set(imp_c)))#7569080 where no click at all

3966
7569080


In [18]:
print len(list((set(imp_c)).intersection(set(imp_nc))))#91560 where both no click and click were present
#Explore these impression ids only to train your model to train click/no_click
train_imp_id=list((set(imp_c)).intersection(set(imp_nc)))

91560


#Get these 3966 impression ids and club them with their attributes to see if any pattern can be identified

In [30]:
imp_c_specific=list(set(imp_c)-set(imp_nc))

In [31]:
impression_sub=impression[impression.impression_id.isin(imp_c_specific)]

In [32]:
print impression.shape
print impression_sub.shape

(33030792, 8)
(5295, 8)


In [33]:
impression_sub.nunique()#We have only 3339 impression ids data in the attribute file

impression_id           3339
geo                       31
site_id                  123
uuid                    3374
adunit_id                239
refrenceUrl             1864
timestamp_impression    5295
uvh                      563
dtype: int64

In [19]:
imp_test=test.impression_id.unique()
imp_train=train.impression_id.unique()
print len(imp_test)
print len(imp_train)

imp_in_train=list(set(imp_train)-set(imp_test))
print "Only in train {}".format(len(imp_in_train))

imp_in_test=list(set(imp_test)-set(imp_train))
print "Only in test {}".format(len(imp_in_test))

imp_common=(list(set(imp_test).intersection(imp_train)))
print "Common in both {}".format(len(list(set(imp_test).intersection(imp_train))))

58879
7664607
Only in train 7656365
Only in test 50637
Common in both 8242


In [ ]:
#This attributes of this subset can be explored to see if we can draw more pattern

In [54]:
geo_sp=impression_sub.geo.unique()
site_id_sp=impression_sub.site_id.unique()
uuid_sp=impression_sub.uuid.unique()
adunit_id_sp=impression_sub.adunit_id.unique()
Url_sp=impression_sub.refrenceUrl.unique()
uvh_sp=impression_sub.uvh.unique()
tim_sp=impression_sub.timestamp_impression.unique()

In [59]:
#Idea is to check if % of click 2 is very high for the above got impression id attributes
# ids=impression.impression_id[impression.geo.isin(geo_sp)]
# train_chk=train[train.impression_id.isin(ids)]
# train_chk.click.value_counts()

1.0    9513503
2.0      76897
Name: click, dtype: int64

In [60]:
ids=impression.impression_id[impression.site_id.isin(site_id_sp)]
train_chk=train[train.impression_id.isin(ids)]
train_chk.click.value_counts()

1.0    9561886
2.0      76703
Name: click, dtype: int64

In [61]:
ids=impression.impression_id[impression.uuid.isin(uuid_sp)]
train_chk=train[train.impression_id.isin(ids)]
train_chk.click.value_counts()

1.0    20711
2.0     4100
Name: click, dtype: int64

In [62]:
ids=impression.impression_id[impression.adunit_id.isin(adunit_id_sp)]
train_chk=train[train.impression_id.isin(ids)]
train_chk.click.value_counts()

1.0    9533226
2.0      76329
Name: click, dtype: int64

In [63]:
ids=impression.impression_id[impression.refrenceUrl.isin(Url_sp)]
train_chk=train[train.impression_id.isin(ids)]
train_chk.click.value_counts()

1.0    3396940
2.0      37933
Name: click, dtype: int64

In [64]:
ids=impression.impression_id[impression.uvh.isin(uvh_sp)]
train_chk=train[train.impression_id.isin(ids)]
train_chk.click.value_counts()

1.0    136889
2.0      1342
Name: click, dtype: int64

In [69]:
item_2=train.item_id[train.click==2]
item_1=train.item_id[train.click==1]
print len(set(item_2))
print len(set(item_1))
print train.item_id.nunique()

3951
6092
6103


In [72]:
item_only_2=list(set(item_2)-set(item_1))
print len(item_only_2)

item_only_1=list(set(item_1)-set(item_2))
print len(item_only_1)

11
2152


In [115]:
item_test=test.item_id.unique()
item_train=train.item_id.unique()
print len(item_test)
print len(item_train)

item_in_train=list(set(item_train)-set(item_test))
print "Only in train {}".format(len(item_in_train))

item_in_test=list(set(item_test)-set(item_train))
print "Only in test {}".format(len(item_in_test))

item_common=(list(set(item_test).intersection(item_train)))
print "Common in both {}".format(len(list(set(item_test).intersection(item_train))))

5167
6104
Only in train 1405
Only in test 468
Common in both 4699


In [118]:
x=(list(set(item_common).intersection(item_only_2)))

In [126]:
train.click.value_counts()

1.0    16918720
2.0      100816
Name: click, dtype: int64

In [127]:
100816-99440

1376

In [116]:
train.click[train.item_id.isin(item_common)].value_counts()

1.0    16573989
2.0       99440
Name: click, dtype: int64

In [ ]:
#Check status of these 4699 item ids

In [111]:
len(set(test.item_id.unique()).intersection(train.item_id.unique()))

4699

In [112]:
len(test.item_id.unique())

5167

In [113]:
4699*1.0/5167

0.9094251983742985

In [128]:
impression.head()

,impression_id,geo,site_id,uuid,adunit_id,refrenceUrl,timestamp_impression,uvh
0,f3b12400-7741-4fc2-8591-37c8e755f127-10uww,144.0,4,adff4c58-1326-4200-a376-01b08faee142-3rpk,210766,https%3A%2F%2Feconomictimes.indiatimes.com%2Fm...,2017-10-25T18:31:39.892+05:30,"0-17232107,17231579,17232759,17237387,17255431"
1,e4a226f3-c1d8-469d-91da-f61b18ee5e5b-10nql,145.0,4,e23b9fd9-0761-45cd-94a3-1440eda65183-10nql,210766,https%3A%2F%2Feconomictimes.indiatimes.com%2Fn...,2017-10-25T18:57:56.494+05:30,NaN
2,9083bb05-072e-4d5c-9072-ccb7069c4070-10wzw,144.0,432,777453b9-c1c9-4db5-af4e-8f28f1936245-10wgg,211156,https%3A%2F%2Fm.economictimes.com%2Fmarkets%2F...,2017-10-25T18:35:26.186+05:30,NaN
3,dd82e108-8360-4e6a-ad8a-1c879b117b18-10ocs,144.0,4,603522ed-2886-40c3-a57b-8a522cfc3d3c-10uww,130014,economictimes.indiatimes.com%2F,2017-10-25T18:34:19.793+05:30,"0-17141460,17162436,17156120,17260598,17261454"
4,1d858bff-db65-4d6d-bed4-af1d2e943829-10ntc,144.0,40,7fa9d303-2930-43da-935e-50542947add6-10o78,195676,https%3A%2F%2Fm.economictimes.com%2Fnews%2Fcom...,2017-10-25T18:02:43.399+05:30,"0-17265399,17265319,17270611,17268606S1-17249871"


In [34]:
train_chk=train[train.impression_id.isin(imp_c_specific)]
train_chk.nunique()

impression_id    3966
item_id           957
click               1
dtype: int64

In [35]:
test_sub=test[test.impression_id.isin(imp_c_specific)]

In [36]:
test_sub.shape

(687, 2)

In [121]:
submission.click[submission.impression_id.isin(imp_c_specific)]=2

/Users/sowmiyanmorri/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [123]:
submission.click[submission.item_id.isin(x)]=2
#=1

/Users/sowmiyanmorri/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [124]:
submission.click.value_counts()

1    188110
2       688
Name: click, dtype: int64

In [81]:
submission.shape

(188798, 3)

In [125]:
submission.to_csv("sub_3.csv",index=False)

In [80]:
np.sqrt((52070.0)/188798)

0.5251641772895308

In [139]:
item_attributes=pd.read_csv("Clean_ItemAttributes.csv")

In [140]:
item_attributes.nunique()

item_id               2108
title                 1834
description            891
timestamp_creation    1439
dtype: int64

In [141]:
item_att=list(set(item_attributes.item_id))

In [142]:
item_train=list(set(train.item_id))
item_test=list(set(test.item_id))

In [143]:
len(list(set(item_att).intersection(set(item_train))))

429

In [144]:
len(list(set(item_att).intersection(set(item_test))))

398

In [146]:
item_train_2=train.item_id[train.click==2]
item_train_1=train.item_id[train.click==1]
print len(set(item_train_2))
print len(set(item_train_1))
print train.item_id.nunique()

item_train_only_2=list(set(item_train_2)-set(item_train_1))
print len(item_only_2)

item_train_only_1=list(set(item_train_1)-set(item_train_2))
print len(item_only_1)

3951
6092
6103
11
2152


In [147]:
len(list(set(item_att).intersection(set(item_train_only_2))))

4